In [ ]:
from geopy.geocoders import Nominatim
import spacy
import pandas as pd
from tqdm import tqdm
from time import sleep
import math
from multiprocess import Pool, Queue
import json

df = pd.read_csv('cleaned_train.csv')
nlp = spacy.load('en_core_web_trf')
locations = list(set(df[df['location'].notna()]['location'].tolist()))
checkpoint_filename = ""

In [ ]:
delta = 235
n_subset = math.ceil(len(locations) / delta)
args = [(locations, i*delta, (i+1)*delta) for i in range(n_subset)]

In [ ]:
def get_zones(locations, start, end):
    index = 0
    zones_found = []
    zones_not_found = []

    try:
        geolocator = Nominatim(user_agent=f"track")

        cicle = None
        if end < len(locations):
            cicle = range(start, end)
        else:
            cicle = range(start, len(locations))

        for i in tqdm(cicle):
            index = i
            sleep(0.05)
            geocode = geolocator.geocode(locations[i])
            if geocode:
                zone = geocode.address.split(',')[-1]
                if zone not in zones_found:
                    zones_found.append(zone)
            else:
                zones_not_found.append(locations[i])
        return zones_found, zones_not_found, end, end
    except Exception as e:
        print(e)
        return zones_found, zones_not_found, index, end

In [ ]:
with Pool(2) as p:
    results = p.starmap(get_zones, args)

In [ ]:
print(results)

In [ ]:
f = open(checkpoint_filename, "a")
checkpoints = []
for z, n_z, s, e in results:
    obj = {
        "zones_found": z,
        "zones_not_found": n_z,
        "start": s,
        "end": e,
    }
    checkpoints.append(obj)
f.write(json.dumps(checkpoints))
f.close()

In [ ]:
f = open(checkpoint_filename, "r")
checkpoint = json.load(f)
total_miss = 0
new_args = []
for c in checkpoint:
    total_miss += (c['end'] - c['start'])
print(total_miss)

In [ ]:
args = []
for c in checkpoint:
    if int(c['start']) != int(c['end']):
        args.append((locations, int(c['start']), int(c['end'])))
args

In [ ]:
for i in checkpoint:
    if i['start'] != i['end']:
        for index in range(i['start'], i['end']):
            print(locations[index])